In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func

In [2]:
# change ip for spark cluster
spark  = SparkSession.builder\
                  .master("spark://172.20.0.9:7077")\
                  .enableHiveSupport()\
                  .appName("WordCount")\
                  .getOrCreate()

In [3]:
#Create a connection to the Spark Cluster
spark = SparkSession.builder.appName("WordCount").getOrCreate()

In [4]:
# Read each line of my book into a dataframe
inputDF = spark.read.text("/usr/local/spark/data/book.txt")

In [5]:
# Split using a regular expression that extracts words
words = inputDF.select(func.explode(func.split(inputDF.value, "\\W+")).alias("word"))
words.filter(words.word != "")

DataFrame[word: string]

In [6]:
# Normalize everything to lowercase
lowercaseWords = words.select(func.lower(words.word).alias("word"))

In [7]:
# Count up the occurrences of each word
wordCounts = lowercaseWords.groupBy("word").count()

In [8]:
# Sort by counts
wordCountsSorted = wordCounts.sort("count")

In [9]:
wordCountsSorted

DataFrame[word: string, count: bigint]

In [10]:
wordCountsSorted.show()

+--------+-----+
|    word|count|
+--------+-----+
| explain|    1|
|      in|    1|
|  cheese|    1|
|     you|    1|
|    will|    1|
|cookbook|    1|
|    make|    1|
|  omelet|    1|
|      to|    1|
|    book|    1|
|      we|    1|
|   hello|    1|
|      is|    1|
|     how|    1|
|       a|    2|
|        |    2|
|    this|    2|
+--------+-----+



In [11]:
wordCountsSorted.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = false)



In [12]:
spark.stop()